In [73]:
import pandas as pd
import math

These data were collected manually from a retail electronic shop named "Samir Electronics", located in Madhabpur, Habiganj. Which is a distributor of a renowed electronic brand in Bangladesh named "Walton".
First step is converting the raw csv files to a proper dataset.

These csv files represents data for an individual year and contains columns as "Date", "Amount", "Model No1", "Model No2", "Model No3", and so on.

In [67]:
# Reading individual csv files and converting to dataframes.

yearlySales1 = pd.read_csv('dataset/Project Dataset/2022_23.csv')
yearlySales2 = pd.read_csv('dataset/Project Dataset/2023_24.csv')

In [68]:
# Contating all dataframes together to a single dataframe.

salesData = pd.concat([yearlySales1, yearlySales2]).reset_index(drop=True)

In [69]:
# Checking which columns do not have any sales data.

emptyColumns = []
for i in salesData.columns:
    if i != "Date" and salesData[i].sum() == 0:
        emptyColumns.append(i)

print(emptyColumns)

['Padestal Fan', 'Wash', 'Beverage Cooler', '1B6']


In [70]:
# Removing unnecessary emtpy columns.

salesData = salesData.drop(columns = emptyColumns)
print(salesData.columns)

Index(['Date', 'Amount', 'Multiplug', 'Mobile P16', 'Mobile N5', 'Mobile H10',
       'Speaker', 'Light Bulb', 'Weight Scale', 'Tornedo Fan',
       ...
       '3J0GDEL DD', '5F3GDEL', '5F3GDEL-DD', 'Air Cooler', '18 A AC',
       '18 C AC', '12 C AC', '24 B AC', '24 C AC', '18 F AC'],
      dtype='object', length=125)


In [78]:
# Making the dataset file.

dataset = "Date,Model,Quantity\n"
for index in salesData.index:
    for column in salesData.columns:
        if column != "Date" and column != "Amount":
            value = salesData[column][index]
            quantity = 0
            if not math.isnan(value):
                quantity = value
            dataset += salesData["Date"][index] + "," + column + "," + str(quantity) + "\n"

f = open("dataset/Project Dataset/sales_data.csv", "w")
f.write(dataset)
f.close()
